**Proyek Ketiga (Akhir) : Image Classification Model Deployment.**

Panduan Project : https://www.dicoding.com/academies/185/tutorials/10629


Computer Generated Images Using Domain Randomization
- Proyek ini menghasilkan model deployment ke format berkas TF-Lite yang dapat di embed di Android dan iOS. 
- Proyek ini menghasilkan model yang bertujuan untuk mengklasifikasikan 4 kelas alias membedakan sepeda(*bikes*), mobil(*cars*), kucing(*cats*), dan anjing(*dogs*).

- Dataset yang digunakan dalam model ini berasal dari ==> [Synthetic Image Dataset (Cats, Dogs, Bikes, Cars)](https://www.kaggle.com/zarkonium/synthetic-image-dataset-cats-dogs-bikes-cars)
- Proyek ini dikerjakan sebagai syarat untuk lulus dalam kelas [Belajar Pengembangan Machine Learning](https://www.dicoding.com/academies/185) di [dicoding](https://www.dicoding.com/)

Nama : Toni Andreas Susanto

Asal : Kota Samarinda, Provinsi Kalimantan Timur

# Import Library yang dibutuhkan

In [ ]:
import pathlib
import zipfile,os
import tensorflow as tf
import keras_preprocessing
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from google.colab import files
from google.colab import drive
from keras.preprocessing import image
from tensorflow.keras.layers import Input
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import ResNet152V2
from keras_preprocessing.image import ImageDataGenerator

# Menyiapkan Dataset

In [ ]:
drive.mount('/content/gdrive')

In [ ]:
# Proses ekstraksi pada file zip
local_zip = '/content/gdrive/MyDrive/dataset/image_dataset.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()

# Mendefinisikan Direktori, Mengetahui Jumlah Gambar dan Menampilkan Sampel Gambar

In [ ]:
folder_bikes = os.path.join('/content/image_dataset/dataset/bikes')
folder_cars = os.path.join('/content/image_dataset/dataset/cars/cars')
folder_cats = os.path.join('/content/image_dataset/dataset/cats/cats')
folder_dogs = os.path.join('/content/image_dataset/dataset/dogs/dogs')
# ==============================================================================
print('Jumlah total foto bikes adalah', len(os.listdir(folder_bikes)))
print('jumlah total foto cars adalah', len(os.listdir(folder_cars)))
print('jumlah total foto cats adalah', len(os.listdir(folder_cats)))
print('jumlah total foto dogs adalah', len(os.listdir(folder_dogs)))

In [ ]:
%matplotlib inline
files_bikes = os.listdir(folder_bikes)
files_cars = os.listdir(folder_cars)
files_cats = os.listdir(folder_cats)
files_dogs = os.listdir(folder_dogs)

nrows = 4
ncols = 5

pic_index = 0 

fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 4)
pic_index += 5

foto_beberapa_bikes = [os.path.join(folder_bikes, fname)
                         for fname in files_bikes[pic_index-5:pic_index]]
foto_beberapa_cars = [os.path.join(folder_cars, fname)
                         for fname in files_cars[pic_index-5:pic_index]]
foto_beberapa_cats = [os.path.join(folder_cats, fname)
                         for fname in files_cats[pic_index-5:pic_index]]
foto_beberapa_dogs = [os.path.join(folder_dogs, fname)
                         for fname in files_dogs[pic_index-5:pic_index]]

for i, img_path in enumerate(foto_beberapa_bikes+foto_beberapa_cars+foto_beberapa_cats+foto_beberapa_dogs):
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off')
  img = mpimg.imread(img_path)
  plt.imshow(img)
plt.show()

# Menerapkan Augmentasi Gambar dan Split Dataset

In [ ]:
base_dir = "/content/image_dataset/dataset"
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=20,
    horizontal_flip=True,
    shear_range=0.2,
    zoom_range=0.2,
    height_shift_range=0.2,
    width_shift_range=0.2,
    fill_mode='nearest',
    validation_split=0.2)

In [ ]:
train_generator = train_datagen.flow_from_directory(
  directory=base_dir,   
  target_size=(150,150), 
  class_mode='categorical',  
  shuffle=True,
  subset='training')

validation_generator = train_datagen.flow_from_directory(
  directory=base_dir,  
  target_size=(150,150),  
  class_mode='categorical',
  shuffle=True,
  subset='validation')

# Membuat Model Sequential

In [ ]:
model = tf.keras.models.Sequential([                             
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.2),  
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.5),  
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')  
])

# Implementasi Callbacks

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.2,
    patience=5, 
    min_lr=1.5e-5
)

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=12,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True
)

# Memilih *Loss, Optimizer dan metrics*

In [ ]:
model.compile(loss ='categorical_crossentropy',
              optimizer = tf.optimizers.Adam(),
              metrics=['accuracy'])

# Pelatihan Model

In [ ]:
history=model.fit(
    train_generator,
    epochs=30,
    batch_size=64,
    verbose=2,
    callbacks = [reduce_lr, early_stop],
    validation_data=validation_generator
)

# Menerapkan *Transfer Learning*

In [ ]:
model = tf.keras.models.Sequential([
    ResNet152V2(weights="imagenet", include_top=False, input_tensor=Input(shape=(150, 150, 3))),
    # tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')  
])
model.layers[0].trainable = False

model.summary()

In [ ]:
model.compile(loss ='categorical_crossentropy',
              optimizer = tf.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
history=model.fit(
    train_generator,
    epochs=30,
    batch_size=64,
    verbose=2,
    callbacks = [reduce_lr, early_stop],
    validation_data=validation_generator)

# Plot Accuracy dan Loss

In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(accuracy))

plt.plot(epochs, accuracy, 'r', label='Accuracy Training')
plt.plot(epochs, val_accuracy, 'b', label='Accuracy Validation')
plt.title('Accuracy Training and Validation')
plt.legend(loc=0)
plt.figure()
plt.show()

plt.plot(epochs, loss, 'r', label='Loss Training')
plt.plot(epochs, val_loss, 'b', label='Loss Validation')
plt.title('Loss Training and Validation')
plt.legend(loc=0)
plt.figure()
plt.show()

# Save format TF-Lite

In [ ]:
# Menyimpan model dalam format SavedModel
export_dir = 'saved_model/'
tf.saved_model.save(model, export_dir)

# Convert SavedModel menjadi vegs.tflite
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()

tflite_model_file = pathlib.Path('vegs.tflite')
tflite_model_file.write_bytes(tflite_model)

# Memprediksi Gambar

Membuat Fungsi untuk Memprediksi Gambar

In [ ]:
def predict_image(image_upload, model = model):
  imge = image_upload
  imge_array = np.asarray(imge)
  imge_array = imge_array*(1/225)
  imge_input = tf.reshape(imge_array, shape = [1, 150, 150, 3])

  predict_array = model.predict(imge_input)[0]
  predict_kelas = np.argmax(model.predict(imge_input))

  if predict_kelas == 0:
      predict_item = 'Bikes'
  elif predict_kelas == 1:
      predict_item = 'Cars'
  elif predict_kelas == 2:
      predict_item = 'Cats'
  elif predict_kelas == 3:
      predict_item = 'Dogs'
  else:
      predict_item = 'Bukan Bikes, Cars, Cats, dan Dogs'

  return predict_item

In [ ]:
def predict():
  uploaded = files.upload()

  for fn in uploaded.keys():
    path = fn
    img = image.load_img(path, target_size=(150,150))
    imgplot = plt.imshow(img)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    img = np.vstack([x])

  kelas = predict_image(img)

  print('\n')
  plt.show()
  print('Gambar yang Anda masukan dikenali sebagai ' + kelas)

Coba Prediksi 4 Gambar Baru

In [ ]:
predict()

In [ ]:
predict()

In [ ]:
predict()

In [ ]:
predict()